In [1]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.signal as sig

In [2]:
# input 

# ground truth data with frames when person was at each end of mat, contains path to yolo data and measured distances 
ground_truth_path = r'C:\Users\mmccu\Box\Brainwalk\Home Video Walking\Megan Project\bw_data_and_code\gait\gait_hc_videos\MM_HC_test_distance\frames_at_end_of_mat_manual.xlsx'
ground_truth_df = pd.read_excel(ground_truth_path, sheet_name = 'Sheet1', engine='openpyxl')

# output 
output_folder = r'C:\Users\mmccu\Box\Brainwalk\Home Video Walking\Megan Project\bw_data_and_code\gait\gait_hc_outputs\MM_HC_test_distance\distance_to_camera'

In [3]:
# stenum paper formula 
# delta_d: depth change of the torso relative to the initial starting depth 
# s_ratio = ratio of current pixel torso size relative to the pixel size or torso at reference depth (current pixel height / reference pixel height)
# d_ref = initial reference depth of person relative to the frontal camera position

# s_ratio = s_i (current pixel height) / s_ref (pixel height at ref distance) 
# delta_d = (d_ref/s_ratio) - d_ref 

In [4]:
# metric options for s_ratio 
    # options - height (neck to shoulder), width (r vs l shoulder), torso area (square root of product of torso ehgith and shoulder width)

# starting with torso height - with turn, width may be inaccurate 
# frame ref = reference distance frame 
# frame_i = current frame 
def calculate_torso_height_ratio(yolo_df, frame_ref, frame_i, fps, video_name):
    yolo_r_shoulder = yolo_df.loc[yolo_df['label'] == 'right_shoulder']
    yolo_r_shoulder.index = yolo_r_shoulder['frame']
    yolo_l_shoulder = yolo_df.loc[yolo_df['label'] == 'left_shoulder']
    yolo_l_shoulder.index = yolo_l_shoulder['frame']
    yolo_r_hip = yolo_df.loc[yolo_df['label'] == 'right_hip']
    yolo_r_hip.index = yolo_r_hip['frame']
    yolo_l_hip = yolo_df.loc[yolo_df['label'] == 'left_hip']
    yolo_l_hip.index = yolo_l_hip['frame']

    # plot both heights 
    fig1, (ax1, ax2) = plt.subplots(2, figsize=(12, 6))
    fig1.suptitle('Hip and Shoulder Y: ' + video_name)
    ax1.plot(-yolo_r_shoulder['Y'], label = 'r_shoulder -Y')
    ax1.plot(-yolo_r_hip['Y'], label = 'r_hip -Y')
    ax1.vlines(x = frame_ref, ymin = min(-yolo_r_hip['Y']), ymax = max(-yolo_r_shoulder['Y']), color = 'black', label = 'frame_ref')
    ax1.vlines(x = frame_i, ymin = min(-yolo_r_hip['Y']), ymax = max(-yolo_r_shoulder['Y']), color = 'red', label = 'frame_i')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    ax2.plot(-yolo_l_shoulder['Y'], label = 'l_shoulder -Y')
    ax2.plot(-yolo_l_hip['Y'], label = 'l_hip -Y')
    ax2.vlines(x = frame_ref, ymin = min(-yolo_r_hip['Y']), ymax = max(-yolo_r_shoulder['Y']), color = 'black', label = 'frame_ref')
    ax2.vlines(x = frame_i, ymin = min(-yolo_r_hip['Y']), ymax = max(-yolo_r_shoulder['Y']), color = 'red', label = 'frame_i')
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig1.savefig(os.path.join(output_folder, video_name + '_hip_shoulder_y.png'))
    plt.close(fig1)
    plt.close()

    # shoulder Y - hip Y at reference frame 
    right_torso_h_pix = abs(yolo_r_shoulder['Y'] - yolo_r_hip['Y'])
    right_torso_h_pix = pd.Series(right_torso_h_pix).rolling(window=10, min_periods=1).mean()
    left_torso_h_pix = abs(yolo_l_shoulder['Y'] - yolo_l_hip['Y'])
    left_torso_h_pix = pd.Series(left_torso_h_pix).rolling(window=10, min_periods=1).mean()
    
    # plot torso heights  
    fig2, ax1 = plt.subplots(figsize=(12, 6))
    fig2.suptitle('Filtered Torso Height: ' + video_name)
    ax1.plot(right_torso_h_pix, label = 'right_height')
    ax1.plot(left_torso_h_pix, label = 'left_height')
    ax1.vlines(x = frame_ref, ymin = min(right_torso_h_pix), ymax = max(right_torso_h_pix), color = 'black', label = 'frame_ref')
    ax1.vlines(x = frame_i, ymin = min(right_torso_h_pix), ymax = max(right_torso_h_pix), color = 'red', label = 'frame_i')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig2.savefig(os.path.join(output_folder, video_name +'torso_height.png'))
    plt.close(fig2)
    plt.close()
    
    # s_ratio = s_i/s_ref  
   # right_s_ratio = right_torso_h_pix_filt/right_torso_h_pix_filt[frame_ref]
   # left_s_ratio = left_torso_h_pix_filt/left_torso_h_pix_filt[frame_ref]
    right_s_ratio = right_torso_h_pix/right_torso_h_pix[frame_ref]
    left_s_ratio = left_torso_h_pix/left_torso_h_pix[frame_ref]
    
    fig3, ax1 = plt.subplots(figsize=(12, 6))
    fig3.suptitle('Height Ratio: ' + video_name)
    ax1.plot(right_s_ratio, label = 'right_height_ratio')
    ax1.plot(left_s_ratio, label = 'left_height_ratio')
    ax1.vlines(x = frame_ref, ymin = min(right_s_ratio), ymax = max(right_s_ratio), color = 'black', label = 'frame_ref')
    ax1.vlines(x = frame_i, ymin = min(right_s_ratio), ymax = max(right_s_ratio), color = 'red', label = 'frame_i')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig3.savefig(os.path.join(output_folder, video_name + '_height_ratio.png'))
    plt.close(fig3)
    plt.close()

    # get size ratio at current frame (frame_i)
    right_s_ratio_i = right_s_ratio[frame_i]
    left_s_ratio_i = left_s_ratio[frame_i]

    return([right_s_ratio, left_s_ratio, right_s_ratio_i, left_s_ratio_i, right_torso_h_pix, left_torso_h_pix])

In [5]:
# idea 1 - using stenum formula - can't be used retrospectively 
    # delta_d = (d_ref/s_ratio) - d_ref
    # d_ref = distance from camera to front end of mat, measured 
    # true delta_d = mat length 
def estimate_delta_d(d_ref, torso_ratio, true_delta_d):
    delta_d_calculated = (d_ref/torso_ratio) - d_ref
    delta_d_error = true_delta_d - delta_d_calculated
    return([delta_d_calculated, delta_d_error])
    

In [6]:
# idea 2 - use turn midpoints frames, expect person to have traveled 17 feet before turning
# could be used retrospectively for in person BW, not for home videos 

def estimate_vel_from_turn_times(turns_df, fps, true_delta_d, torso_h, video_name): 
    
    turn_midpoints = turns_df['turn_midpoint']
    time_btwn_turn_midpoints = np.diff(turn_midpoints)
    velocity_from_turns = true_delta_d / (time_btwn_turn_midpoints/fps)
    
    fig4, ax1 = plt.subplots()
    ax1.plot(torso_h, label = 'torso_height', color = 'black')
    ax1.scatter(x = turn_midpoints, y = torso_h[turn_midpoints], color = 'blue', label = 'turn_midpoints')
    ax1.legend()
    fig4.savefig(os.path.join(output_folder, video_name + '_turn_midpoints_by_height.png'))
    plt.close(fig4)
    plt.close()

    return(velocity_from_turns)

In [14]:
# for each video (row), calculate change in distance from size ratio 
# save values and errors in .csv file 

#output .csvs  
dist_from_ratio_df = pd.DataFrame(index = range(len(ground_truth_df)), 
                                columns = ['video', 
                                           'true_delta_d',
                                           'true_d_ref',
                                           'frame_ref_1', 
                                           'frame_i_1', 
                                           'estimated_delta_d_1', 
                                           'estimated_delta_d_1_error',
                                           'estimated_velocity_1_mps',
                                           'frame_ref_2', 
                                           'frame_i_2', 
                                           'estimated_delta_d_2',
                                           'estimated_delta_d_2_error',
                                           'estimated_velocity_2_mps'])

vel_from_turn_time = pd.DataFrame(index = range(len(ground_truth_df)),
                                  columns = ['video',
                                             'estimated_velocity_1_mps',
                                             'estimated_velocity_2_mps'])
                                                          
for video_i, current_video in enumerate(ground_truth_df['video']):
    
    dist_from_ratio_df.loc[video_i, "video"] = current_video
    true_delta_d = ground_truth_df['mat_length_ft'].iloc[video_i] / (3.281) # convert feet to meters 
    dist_from_ratio_df.loc[video_i, "true_delta_d"] = true_delta_d
    true_d_ref = ground_truth_df['dis_cam_to_front_mat_ft'].iloc[video_i]  / (3.281)
    dist_from_ratio_df.loc[video_i, "true_d_ref"] = true_d_ref
    yolo_path = ground_truth_df['yolo_full_path'].iloc[video_i] 
    yolo_df = pd.read_csv(yolo_path, index_col =0)
    fps = 30 # know my phone and BW camera is 30, need to update if using other cameras 
    turn_times_path = ground_truth_df['turn_times_path'].iloc[video_i]
    turn_times_df = pd.read_csv(turn_times_path, index_col = 0)

    # estimate distance change using size ratio change 
    if ~np.isnan(ground_truth_df['front_end_mat_1'].iloc[video_i]) & ~np.isnan(ground_truth_df['far_end_mat_1'].iloc[video_i]):
        print('walk 1')
        # calculate values
        frame_ref = int(ground_truth_df['front_end_mat_1'].iloc[video_i])
        frame_i = int(ground_truth_df['far_end_mat_1'].iloc[video_i])
        torso_height_results_1 = calculate_torso_height_ratio(yolo_df, frame_ref, frame_i, fps, current_video)
        delta_d_results_1 =  estimate_delta_d(true_d_ref, torso_height_results_1[2], true_delta_d)
        #d_ref_results_1 = estimate_d_ref(true_delta_d, torso_height_results_1[2], true_d_ref)
        print('estimate dist')
        print(frame_ref, frame_i, torso_height_results_1[2], delta_d_results_1)
        
        # save results in df 
        dist_from_ratio_df.loc[video_i, "frame_ref_1"] = frame_ref
        dist_from_ratio_df.loc[video_i, "frame_i_1"] = frame_i
        dist_from_ratio_df.loc[video_i, "estimated_delta_d_1"] = delta_d_results_1[0]
        dist_from_ratio_df.loc[video_i, 'estimated_delta_d_1_error'] = delta_d_results_1[1]
        dist_from_ratio_df.loc[video_i, 'estimated_velocity_1_mps'] = delta_d_results_1[0] / ((frame_i - frame_ref)/fps) # m/s
        #dist_from_ratio_df.loc[video_i, "estimated_d_ref_1"] = d_ref_results_1[0]
       # dist_from_ratio_df.loc[video_i, 'estimated_d_ref_1_error'] = d_ref_results_1[1]
        

    # if there is data for second walk from camera 
    if ~np.isnan(ground_truth_df['front_end_mat_2'].iloc[video_i]) & ~np.isnan(ground_truth_df['far_end_mat_2'].iloc[video_i]):
        print('walk 2')
        # calculate values 
        frame_ref = int(ground_truth_df['front_end_mat_2'].iloc[video_i])
        frame_i = int(ground_truth_df['far_end_mat_2'].iloc[video_i])
        torso_height_results_2 = calculate_torso_height_ratio(yolo_df, frame_ref, frame_i, fps, current_video)
        delta_d_results_2 = estimate_delta_d(true_d_ref, torso_height_results_2[2], true_delta_d)
       # d_ref_results_2 = estimate_d_ref(true_delta_d, torso_height_results_2[2], true_d_ref)
        print('estimate dist')
        print(frame_ref, frame_i, torso_height_results_2[2], delta_d_results_2)

        # save results in df 
        dist_from_ratio_df.loc[video_i, "frame_ref_2"] = frame_ref
        dist_from_ratio_df.loc[video_i, "frame_i_2"] = frame_i
        dist_from_ratio_df.loc[video_i, "estimated_delta_d_2"] = delta_d_results_2[0]
        dist_from_ratio_df.loc[video_i, 'estimated_delta_d_2_error'] = delta_d_results_2[1]
        dist_from_ratio_df.loc[video_i, 'estimated_velocity_2_mps'] = delta_d_results_2[0] / ((frame_i - frame_ref)/fps) # m/s
        #dist_from_ratio_df.loc[video_i, "estimated_d_ref_2"] = d_ref_results_2[0]
        #dist_from_ratio_df.loc[video_i, 'estimated_d_ref_2_error'] = d_ref_results_2[1]

    # estimate velocity with known distance travled and turn midpoints 
    velocity_from_turns = estimate_vel_from_turn_times(turn_times_df, fps, true_delta_d, torso_height_results_1[4], current_video)
    
    vel_from_turn_time.loc[video_i, 'video'] = current_video
    vel_from_turn_time.loc[video_i,'estimated_velocity_1_mps'] = velocity_from_turns[0]
    vel_from_turn_time.loc[video_i,'estimated_velocity_2_mps'] = velocity_from_turns[1]



# save .csv 
dist_from_ratio_df.to_csv(os.path.join(output_folder, 'delta_d_and_vel_from_ratio.csv'))
vel_from_turn_time.to_csv(os.path.join(output_folder, 'vel_from_turn_time.csv'))

walk 1
estimate dist
1 160 0.3599208952604652 [4.878236693705038, 0.30311045655402946]
walk 2
estimate dist
289 452 0.4070181269592236 [3.996353943149515, 1.1849932071095521]
walk 1
estimate dist
1 168 0.36747064845265814 [4.7216556017799265, 0.4596915484791406]
walk 2
estimate dist
333 507 0.4050133912140999 [4.029712732344931, 1.1516344179141358]
walk 1
estimate dist
41 230 0.262694424608143 [6.950464056446975, -1.7691169061879082]
walk 2
estimate dist
389 546 0.2898457365209478 [6.067404110846963, -0.8860569605878963]


In [13]:
vel_from_turn_time

,video,estimated_velocity_1_mps,estimated_velocity_2_mps
0,MM_HC_17ft_gait_vertical_right,0.914355,0.868382
1,MM_HC_17ft_gait_vertical_right,0.914355,0.868382
2,MM_HC_17ft_gait_vertical_right,0.914355,0.868382


In [9]:
# other ideas 
# double check formula 
# torso area? not sure how much width affects 
# filter 

In [10]:
# idea 2 - know distance traveled -> use ratio at front and back of mat (17 ft) to make ratio 
#-> get distance from camera at front end of mat 

# delta_d = (d_ref/s_ratio) - d_ref -> solve for d_ref -> d_ref = (-s_ratio * delta_d) / (s_ratio - 1)
# delta_d = distance traveled between frame_ref and frame_i, math length 
# true_d_ref = true distance from camera to front of mat, measured 
def estimate_d_ref(delta_d, torso_ratio, true_d_ref):
    d_ref_calculated = (-torso_ratio * delta_d) / (torso_ratio - 1)
    d_ref_error = true_d_ref - d_ref_calculated
    return([d_ref_calculated, d_ref_error])